In [78]:
set.seed(123)

In [79]:
getwd()
setwd("/liulab/galib/dlbcl_manuscript/")
library(rBCS)
library(tidyverse)
library(Seurat)
library(harmony)
library(viridis)
library(RColorBrewer)
library(Polychrome)
PurpleAndYellow()
library(ComplexHeatmap)
library(devtools)
library(presto)
library(dplyr)
library(ggplot2)
library(ggpubr)
library(readxl)
source("./scripts/scplot.R")

[1] "/liulab/galib/dlbcl_manuscript"

[1] "#FF00FF" "#F400F4" "#EA00EA" "#DF00DF" "#D500D5" "#CA00CA" "#BF00BF"
 [8] "#B500B5" "#AA00AA" "#9F009F" "#950095" "#8A008A" "#800080" "#750075"
[15] "#6A006A" "#600060" "#550055" "#4A004A" "#400040" "#350035" "#2B002B"
[22] "#200020" "#150015" "#0B000B" "#000000" "#000000" "#0B0B00" "#151500"
[29] "#202000" "#2B2B00" "#353500" "#404000" "#4A4A00" "#555500" "#606000"
[36] "#6A6A00" "#757500" "#808000" "#8A8A00" "#959500" "#9F9F00" "#AAAA00"
[43] "#B5B500" "#BFBF00" "#CACA00" "#D4D400" "#DFDF00" "#EAEA00" "#F4F400"
[50] "#FFFF00"

In [80]:
cd3_pos_cd8_neg<- readRDS("./data/objects/cd3_pos_cd8_neg_final.rds")
cd3_pos_cd4_neg<- readRDS("./data/objects/cd3_pos_cd4_neg_final.rds")
B_cell<- readRDS("./data/objects/B_cell_final.rds")
non_B_non_T<- readRDS("data/objects/non_B_non_T_final.rds")

In [90]:
legend_dot_size = unit(2,"mm")

lgd = Legend(labels = c(0, 10, 25, 50, 75, 100),
           title = "percentage",
           type = "points",
        graphics = list(
    function(x, y, w, h) grid.circle(x, y, r= sqrt(0/100) * legend_dot_size,
                                    gp = gpar(fill = "black")),
    function(x, y, w, h) grid.circle(x, y, r= sqrt(10/100) * legend_dot_size,
                                    gp = gpar(fill = "black")),
    function(x, y, w, h) grid.circle(x, y, r= sqrt(25/100) * legend_dot_size,
                                    gp = gpar(fill = "black")),
    function(x, y, w, h) grid.circle(x, y, r= sqrt(50/100) * legend_dot_size,
                                    gp = gpar(fill = "black")),
    function(x, y, w, h) grid.circle(x, y, r= sqrt(75/100) * legend_dot_size,
                                    gp = gpar(fill = "black")),
    function(x, y, w, h) grid.circle(x, y, r= sqrt(100/100) * legend_dot_size,
                                    gp = gpar(fill = "black"))            
            )
     )

## B_cell space

In [82]:
B_cell_genes<- read_tsv("./data/B_cell_genes_to_plot.tsv", col_names = FALSE)  %>% 
    pull(X1)

B_cell_genes<- str_to_title(B_cell_genes)
B_cell_genes<- B_cell_genes  %>% gsub(pattern = "Aw", replacement = "AW")
B_cell_genes<- B_cell_genes  %>% gsub(pattern = 'C130026l21rik', replacement = 'C130026L21Rik')
B_cell_genes<- B_cell_genes  %>% gsub(pattern = 'D10wsu102e', replacement = 'D10Wsu102e')
B_cell_genes<- B_cell_genes  %>% gsub(pattern = 'Ighg2', replacement = 'Ighg2b')
B_cell_genes<- B_cell_genes  %>% gsub(pattern = 'Cd21', replacement = 'Cr2')
B_cell_genes<- B_cell_genes  %>% gsub(pattern = 'Ac133103.1', replacement = 'Sp140')
B_cell_genes<- B_cell_genes  %>% gsub(pattern = 'Tubb', replacement = 'Tubb5')
B_cell_genes<- B_cell_genes  %>% gsub(pattern = 'Igkv 16-104', replacement = 'Igkv16-104')
B_cell_genes<- B_cell_genes  %>% gsub(pattern = 'Ighv 1-52', replacement = 'Ighv1-52')
B_cell_genes<- B_cell_genes  %>% gsub(pattern = 'Ighv 1-52', replacement = 'Ighv1-52')
B_cell_genes<- B_cell_genes  %>% gsub(pattern = 'Pdcd2l2', replacement = 'Pdcd2l')

B_cell_genes[!B_cell_genes %in% rownames(B_cell)]

B_cell_genes<- B_cell_genes  %>% unique()

Rows: 120 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (1): X1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


character(0)

In [83]:
B_cell_col<- B_cell@meta.data  %>% select(new_annotation, HEX_code)  %>% distinct()

In [84]:
B_cell_cluster_order<- c(8,7,4,5,1,22,18,21,19,16,3,27,2,17,25,11,10,14,6,13,12,24,23,20,9,0,15)
B_cell$seurat_clusters<- factor(B_cell$seurat_clusters, levels = B_cell_cluster_order)
cluster_id<- B_cell@meta.data  %>% arrange(seurat_clusters)  %>% distinct(new_annotation)  %>% pull(new_annotation)
B_cell_colors<- B_cell@meta.data  %>% arrange(seurat_clusters)  %>% distinct(HEX_code)  %>% pull(HEX_code)

### All samples

In [107]:
B_cell_mtx<- GetMatrixFromSeurat(B_cell, B_cell_genes, group = "new_annotation")

column_ha <- HeatmapAnnotation(foo = anno_text(gsub("_.*", "", cluster_id), location = 0.5, just = "center", 
                    gp = gpar(fill = setNames(B_cell_colors, gsub("_.*", "", cluster_id)), 
                              col = "white", border = NA)), show_annotation_name = FALSE, show_legend = FALSE)

col_fun = circlize::colorRamp2(c(-2,0,2), viridis::viridis(3))

p<- MakeClusterDotPlot(exp_mat = B_cell_mtx$exp_mat, 
                       percent_mat = B_cell_mtx$percent_mat,
                       cluster_id = cluster_id,
                       column_ha = column_ha, col_fun = col_fun,
                       column_title_side = "bottom",
                       column_title = "B cell Space", 
                       row_km = 6,
                       row_names_font_size = 8, 
                       legend_dot_size = unit(2, "mm"),
                       column_names_side = "top",
                       show_column_dend = FALSE,
                       show_row_dend = FALSE, 
                       cluster_columns = FALSE)

pdf("results/figures/5_B_cell_dotplot_final_annotation.pdf", width = 7, height = 15)

draw(p, annotation_legend_list = lgd)

dev.off()

png 
  2

### Dotplot by genotype

In [109]:
WT_cells<- rownames(B_cell@meta.data  %>% filter(genotype == "WT"))
cd70_cells<- rownames(B_cell@meta.data  %>% filter(genotype == "CD70-/-"))
single_cells<- rownames(B_cell@meta.data  %>% filter(genotype == "Bcl6tg/+"))
double_cells<- rownames(B_cell@meta.data  %>% filter(genotype == "CD70-/-;Bcl6tg/+"))

B_cell_mtx<- GetMatrixFromSeurat2(B_cell, B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_WT<- GetMatrixFromSeurat2(B_cell[,WT_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_cd70<- GetMatrixFromSeurat2(B_cell[,cd70_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_single<- GetMatrixFromSeurat2(B_cell[,single_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_double<- GetMatrixFromSeurat2(B_cell[,double_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_combined<- cbind(B_cell_mtx_WT$exp_mat, B_cell_mtx_cd70$exp_mat, 
                                 B_cell_mtx_single$exp_mat, B_cell_mtx_double$exp_mat)

B_cell_combined_scaled<- t(apply(B_cell_combined, 1, scale))
B_cell_combined_scaled<- B_cell_combined_scaled  %>% replace(is.na(.),0)

ncol(B_cell_combined_scaled)
B_cell_mtx_WT_scaled<-  B_cell_combined_scaled[, 1:27]
B_cell_mtx_cd70_scaled<-  B_cell_combined_scaled[, 28:54]
B_cell_mtx_single_scaled<-  B_cell_combined_scaled[, 55:81]
B_cell_mtx_double_scaled<-  B_cell_combined_scaled[, 82:108]

colnames(B_cell_mtx_WT_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_cd70_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_single_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_double_scaled)<- colnames(B_cell_mtx$exp_mat)

[1] 108

In [110]:
column_ha <- HeatmapAnnotation(foo = anno_text(gsub("_.*", "", cluster_id), 
                                               location = 0.5, just = "center", 
                                               gp = gpar(fill = setNames(B_cell_colors, 
                                                                         gsub("_.*", "", cluster_id)), 
                                                         col = "white", border = NA)), 
                               show_annotation_name = FALSE, show_legend = FALSE)

col_fun = circlize::colorRamp2(c(-2,0,2), viridis::viridis(3))

In [113]:
pdf("./results/figures/5_B_cell_dotplot_final_annotation_by_genotype.pdf", width = 30, height = 15)

hp1<- MakeClusterDotPlot(exp_mat = B_cell_mtx_WT_scaled, 
               percent_mat = B_cell_mtx_WT$percent_mat, 
               col_fun = col_fun,
               cluster_id = cluster_id,              
               column_title = "WT",
               column_title_side = "bottom",
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha,
               legend_dot_size = unit(2, "mm"),
               show_row_dend =FALSE,
               show_column_dend = FALSE,
               column_names_side = "top",
               cluster_columns = FALSE)

hp2<- MakeClusterDotPlot(exp_mat = B_cell_mtx_cd70_scaled, 
               percent_mat = B_cell_mtx_cd70$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "CD70-/-",
               column_title_side = "bottom",
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha,
               legend_dot_size = unit(2, "mm"),
               row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               column_names_side = "top",  cluster_columns = FALSE)

hp3<- MakeClusterDotPlot(exp_mat = B_cell_mtx_single_scaled, 
               percent_mat = B_cell_mtx_single$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "Bcl6tg/+",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha,
               legend_dot_size = unit(2, "mm"),
               row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               column_names_side = "top",  cluster_columns = FALSE)

hp4<- MakeClusterDotPlot(exp_mat = B_cell_mtx_double_scaled, 
               percent_mat = B_cell_mtx_double$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "CD70-/-;Bcl6tg/+",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha,
               legend_dot_size = unit(2, "mm"),
               row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               column_names_side = "top",  cluster_columns = FALSE)

draw(hp1 + hp2 + hp3 + hp4 , annotation_legend_list = lgd, ht_gap = unit(1, "cm") )
dev.off()

Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”
Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”
Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”


png 
  2

### Dotplot by age

In [114]:
n_6mos_cells<- rownames(B_cell@meta.data  %>% filter(age == "6mos"))
n_14mos_cells<- rownames(B_cell@meta.data  %>% filter(age == "14mos"))
n_18mos_cells<- rownames(B_cell@meta.data  %>% filter(age == "18mos"))
sick_cells<- rownames(B_cell@meta.data  %>% filter(age == "sick"))

B_cell_mtx<- GetMatrixFromSeurat2(B_cell, B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_6mos<- GetMatrixFromSeurat2(B_cell[,n_6mos_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_14mos<- GetMatrixFromSeurat2(B_cell[,n_14mos_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_18mos<- GetMatrixFromSeurat2(B_cell[,n_18mos_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_sick<- GetMatrixFromSeurat2(B_cell[,sick_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_combined<- cbind(B_cell_mtx_6mos$exp_mat, B_cell_mtx_14mos$exp_mat, 
                                 B_cell_mtx_18mos$exp_mat, B_cell_mtx_sick$exp_mat)

B_cell_combined_scaled<- t(apply(B_cell_combined, 1, scale))
B_cell_combined_scaled<- B_cell_combined_scaled  %>% replace(is.na(.),0)

ncol(B_cell_combined_scaled)
B_cell_mtx_6mos_scaled<-  B_cell_combined_scaled[, 1:27]
B_cell_mtx_14mos_scaled<-  B_cell_combined_scaled[, 28:54]
B_cell_mtx_18mos_scaled<-  B_cell_combined_scaled[, 55:81]
B_cell_mtx_sick_scaled<-  B_cell_combined_scaled[, 82:108]

colnames(B_cell_mtx_6mos_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_14mos_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_18mos_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_sick_scaled)<- colnames(B_cell_mtx$exp_mat)

[1] 108

In [115]:
column_ha <- HeatmapAnnotation(foo = anno_text(gsub("_.*", "", cluster_id), 
                                               location = 0.5, just = "center", 
                                               gp = gpar(fill = setNames(B_cell_colors, 
                                                                         gsub("_.*", "", cluster_id)), 
                                                         col = "white", border = NA)), 
                               show_annotation_name = FALSE, show_legend = FALSE)

col_fun = circlize::colorRamp2(c(-2,0,2), viridis::viridis(3))

In [117]:
pdf("./results/figures/5_B_cell_dotplot_final_annotation_by_age.pdf", width = 30, height = 15)

hp1<- MakeClusterDotPlot(exp_mat = B_cell_mtx_6mos_scaled, 
           percent_mat = B_cell_mtx_6mos$percent_mat,
           col_fun = col_fun, cluster_id = cluster_id,
           column_title = "6mos",
           column_title_side = "bottom",
           row_km = 6,
           row_names_font_size = 8, 
           column_ha = column_ha,
           legend_dot_size = unit(2, "mm"),
           show_row_dend =FALSE,
           show_column_dend = FALSE,
           column_names_side = "top", cluster_columns=FALSE)

hp2<- MakeClusterDotPlot(exp_mat = B_cell_mtx_14mos_scaled, 
               percent_mat = B_cell_mtx_14mos$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "14mos",
               column_title_side = "bottom",
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha,
               legend_dot_size = unit(2, "mm"),
               row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               column_names_side = "top", cluster_columns=FALSE)

hp3<- MakeClusterDotPlot(exp_mat = B_cell_mtx_18mos_scaled, 
               percent_mat = B_cell_mtx_18mos$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "18mos",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha,
               legend_dot_size = unit(2, "mm"),
               row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               column_names_side = "top", cluster_columns=FALSE)

hp4<- MakeClusterDotPlot(exp_mat = B_cell_mtx_sick_scaled, 
               percent_mat = B_cell_mtx_sick$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "sick",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha,
               legend_dot_size = unit(2, "mm"),
               row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               column_names_side = "top", cluster_columns=FALSE)

draw(hp1 + hp2 + hp3 + hp4 , annotation_legend_list = lgd, ht_gap = unit(1, "cm") )
dev.off()

Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”
Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”
Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”
Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); column_order(ht)`.”


png 
  2

### By age and genotype 14 class

- 6mos (WT, CD70-/-, Bcl6tg/+, CD70-/-;Bcl6tg/+)

In [128]:
n_6mos_WT_cells<- rownames(B_cell@meta.data  %>% filter(age == "6mos", genotype == "WT"))
n_6mos_cd70_cells<- rownames(B_cell@meta.data  %>% filter(age == "6mos", genotype == "CD70-/-"))
n_6mos_single_cells<- rownames(B_cell@meta.data  %>% filter(age == "6mos", genotype == "Bcl6tg/+"))
n_6mos_double_cells<- rownames(B_cell@meta.data  %>% filter(age == "6mos", genotype == "CD70-/-;Bcl6tg/+"))

B_cell_mtx<- GetMatrixFromSeurat2(B_cell, B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_6mos_WT<- GetMatrixFromSeurat2(B_cell[,n_6mos_WT_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_6mos_cd70<- GetMatrixFromSeurat2(B_cell[,n_6mos_cd70_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_6mos_single<- GetMatrixFromSeurat2(B_cell[,n_6mos_single_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_6mos_double<- GetMatrixFromSeurat2(B_cell[,n_6mos_double_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_combined<- cbind(B_cell_mtx_6mos_WT$exp_mat,
                        B_cell_mtx_6mos_cd70$exp_mat,
                        B_cell_mtx_6mos_single$exp_mat,
                        B_cell_mtx_6mos_double$exp_mat)

B_cell_combined_scaled<- t(apply(B_cell_combined, 1, scale))
B_cell_combined_scaled<- B_cell_combined_scaled  %>% replace(is.na(.),0)

ncol(B_cell_combined_scaled)
B_cell_mtx_6mos_WT_scaled<-  B_cell_combined_scaled[, 1:27]
B_cell_mtx_6mos_cd70_scaled<-  B_cell_combined_scaled[, 28:54]
B_cell_mtx_6mos_single_scaled<-  B_cell_combined_scaled[, 55:81]
B_cell_mtx_6mos_double_scaled<-  B_cell_combined_scaled[, 82:108]

colnames(B_cell_mtx_6mos_WT_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_6mos_cd70_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_6mos_single_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_6mos_double_scaled)<- colnames(B_cell_mtx$exp_mat)

pdf("./results/figures/5_B_cell_dotplot_final_annotation_6mos_by_genotype.pdf", width = 30, height = 15)

hp1<- MakeClusterDotPlot(exp_mat = B_cell_mtx_6mos_WT_scaled, 
               percent_mat = B_cell_mtx_6mos_WT$percent_mat, 
               col_fun = col_fun,
               cluster_id = cluster_id,              
               column_title = "6mos WT",
               column_title_side = "bottom",
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha,
               legend_dot_size = unit(2, "mm"),
               show_row_dend =FALSE,
               show_column_dend = FALSE,
               column_names_side = "top",
               cluster_columns = FALSE)

hp2<- MakeClusterDotPlot(exp_mat = B_cell_mtx_6mos_cd70_scaled, 
               percent_mat = B_cell_mtx_6mos_cd70$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "6mos CD70-/-",
               column_title_side = "bottom",
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha, row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               legend_dot_size = unit(2, "mm"),
               column_names_side = "top",  cluster_columns = FALSE)

hp3<- MakeClusterDotPlot(exp_mat = B_cell_mtx_6mos_single_scaled, 
               percent_mat = B_cell_mtx_6mos_single$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "6mos Bcl6tg/+",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha, row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               legend_dot_size = unit(2, "mm"),
               column_names_side = "top",  cluster_columns = FALSE)

hp4<- MakeClusterDotPlot(exp_mat = B_cell_mtx_6mos_double_scaled, 
               percent_mat = B_cell_mtx_6mos_double$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "6mos CD70-/-;Bcl6tg/+",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha, row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               legend_dot_size = unit(2, "mm"),
               column_names_side = "top",  cluster_columns = FALSE)

draw(hp1 + hp2 + hp3 + hp4 , annotation_legend_list = lgd, ht_gap = unit(1, "cm") )
dev.off()

[1] 108

Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”
Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”
Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”


png 
  2

- 14mos (WT, CD70-/-, Bcl6tg/+, CD70-/-;Bcl6tg/+)

In [135]:
n_14mos_WT_cells<- rownames(B_cell@meta.data  %>% filter(age == "14mos", genotype == "WT"))
n_14mos_cd70_cells<- rownames(B_cell@meta.data  %>% filter(age == "14mos", genotype == "CD70-/-"))
n_14mos_single_cells<- rownames(B_cell@meta.data  %>% filter(age == "14mos", genotype == "Bcl6tg/+"))
n_14mos_double_cells<- rownames(B_cell@meta.data  %>% filter(age == "14mos", genotype == "CD70-/-;Bcl6tg/+"))

B_cell_mtx<- GetMatrixFromSeurat2(B_cell, B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_14mos_WT<- GetMatrixFromSeurat2(B_cell[,n_14mos_WT_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_14mos_cd70<- GetMatrixFromSeurat2(B_cell[,n_14mos_cd70_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_14mos_single<- GetMatrixFromSeurat2(B_cell[,n_14mos_single_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_14mos_double<- GetMatrixFromSeurat2(B_cell[,n_14mos_double_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_combined<- cbind(B_cell_mtx_14mos_WT$exp_mat,
                        B_cell_mtx_14mos_cd70$exp_mat,
                        B_cell_mtx_14mos_single$exp_mat,
                        B_cell_mtx_14mos_double$exp_mat)

B_cell_combined_scaled<- t(apply(B_cell_combined, 1, scale))
B_cell_combined_scaled<- B_cell_combined_scaled  %>% replace(is.na(.),0)

ncol(B_cell_combined_scaled)
B_cell_mtx_14mos_WT_scaled<-  B_cell_combined_scaled[, 1:27]
B_cell_mtx_14mos_cd70_scaled<-  B_cell_combined_scaled[, 28:54]
B_cell_mtx_14mos_single_scaled<-  B_cell_combined_scaled[, 55:81]
B_cell_mtx_14mos_double_scaled<-  B_cell_combined_scaled[, 82:108]

colnames(B_cell_mtx_14mos_WT_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_14mos_cd70_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_14mos_single_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_14mos_double_scaled)<- colnames(B_cell_mtx$exp_mat)

pdf("./results/figures/5_B_cell_dotplot_final_annotation_14mos_by_genotype.pdf", width = 30, height = 15)

hp1<- MakeClusterDotPlot(exp_mat = B_cell_mtx_14mos_WT_scaled, 
               percent_mat = B_cell_mtx_14mos_WT$percent_mat, 
               col_fun = col_fun,
               cluster_id = cluster_id,              
               column_title = "14mos WT",
               column_title_side = "bottom",
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha,
               legend_dot_size = unit(2, "mm"),
               show_row_dend =FALSE,
               show_column_dend = FALSE,
               column_names_side = "top",
               cluster_columns = FALSE)

hp2<- MakeClusterDotPlot(exp_mat = B_cell_mtx_14mos_cd70_scaled, 
               percent_mat = B_cell_mtx_14mos_cd70$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "14mos CD70-/-",
               column_title_side = "bottom",
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha, row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               legend_dot_size = unit(2, "mm"),
               column_names_side = "top",  cluster_columns = FALSE)

hp3<- MakeClusterDotPlot(exp_mat = B_cell_mtx_14mos_single_scaled, 
               percent_mat = B_cell_mtx_14mos_single$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "14mos Bcl6tg/+",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha, row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               legend_dot_size = unit(2, "mm"),
               column_names_side = "top",  cluster_columns = FALSE)

hp4<- MakeClusterDotPlot(exp_mat = B_cell_mtx_14mos_double_scaled, 
               percent_mat = B_cell_mtx_14mos_double$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "14mos CD70-/-;Bcl6tg/+",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha, row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               legend_dot_size = unit(2, "mm"),
               column_names_side = "top",  cluster_columns = FALSE)

draw(hp1 + hp2 + hp3 + hp4 , annotation_legend_list = lgd, ht_gap = unit(1, "cm") )
dev.off()

[1] 108

Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”
Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”
Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”


png 
  2

- 18mos (WT, CD70-/-, Bcl6tg/+, CD70-/-;Bcl6tg/+)

In [136]:
n_18mos_WT_cells<- rownames(B_cell@meta.data  %>% filter(age == "18mos", genotype == "WT"))
n_18mos_cd70_cells<- rownames(B_cell@meta.data  %>% filter(age == "18mos", genotype == "CD70-/-"))
n_18mos_single_cells<- rownames(B_cell@meta.data  %>% filter(age == "18mos", genotype == "Bcl6tg/+"))
n_18mos_double_cells<- rownames(B_cell@meta.data  %>% filter(age == "18mos", genotype == "CD70-/-;Bcl6tg/+"))

B_cell_mtx<- GetMatrixFromSeurat2(B_cell, B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_18mos_WT<- GetMatrixFromSeurat2(B_cell[,n_18mos_WT_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_18mos_cd70<- GetMatrixFromSeurat2(B_cell[,n_18mos_cd70_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_18mos_single<- GetMatrixFromSeurat2(B_cell[,n_18mos_single_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_18mos_double<- GetMatrixFromSeurat2(B_cell[,n_18mos_double_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_combined<- cbind(B_cell_mtx_18mos_WT$exp_mat,
                        B_cell_mtx_18mos_cd70$exp_mat,
                        B_cell_mtx_18mos_single$exp_mat,
                        B_cell_mtx_18mos_double$exp_mat)

B_cell_combined_scaled<- t(apply(B_cell_combined, 1, scale))
B_cell_combined_scaled<- B_cell_combined_scaled  %>% replace(is.na(.),0)

ncol(B_cell_combined_scaled)
B_cell_mtx_18mos_WT_scaled<-  B_cell_combined_scaled[, 1:27]
B_cell_mtx_18mos_cd70_scaled<-  B_cell_combined_scaled[, 28:54]
B_cell_mtx_18mos_single_scaled<-  B_cell_combined_scaled[, 55:81]
B_cell_mtx_18mos_double_scaled<-  B_cell_combined_scaled[, 82:108]

colnames(B_cell_mtx_18mos_WT_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_18mos_cd70_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_18mos_single_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_18mos_double_scaled)<- colnames(B_cell_mtx$exp_mat)

pdf("./results/figures/5_B_cell_dotplot_final_annotation_18mos_by_genotype.pdf", width = 30, height = 15)

hp1<- MakeClusterDotPlot(exp_mat = B_cell_mtx_18mos_WT_scaled, 
               percent_mat = B_cell_mtx_18mos_WT$percent_mat, 
               col_fun = col_fun,
               cluster_id = cluster_id,              
               column_title = "18mos WT",
               column_title_side = "bottom",
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha,
               legend_dot_size = unit(2, "mm"),
               show_row_dend =FALSE,
               show_column_dend = FALSE,
               column_names_side = "top",
               cluster_columns = FALSE)

hp2<- MakeClusterDotPlot(exp_mat = B_cell_mtx_18mos_cd70_scaled, 
               percent_mat = B_cell_mtx_18mos_cd70$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "18mos CD70-/-",
               column_title_side = "bottom",
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha, row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               legend_dot_size = unit(2, "mm"),
               column_names_side = "top",  cluster_columns = FALSE)

hp3<- MakeClusterDotPlot(exp_mat = B_cell_mtx_18mos_single_scaled, 
               percent_mat = B_cell_mtx_18mos_single$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "18mos Bcl6tg/+",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha, row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               legend_dot_size = unit(2, "mm"),
               column_names_side = "top",  cluster_columns = FALSE)

hp4<- MakeClusterDotPlot(exp_mat = B_cell_mtx_18mos_double_scaled, 
               percent_mat = B_cell_mtx_18mos_double$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "18mos CD70-/-;Bcl6tg/+",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha, row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               legend_dot_size = unit(2, "mm"),
               column_names_side = "top",  cluster_columns = FALSE)

draw(hp1 + hp2 + hp3 + hp4 , annotation_legend_list = lgd, ht_gap = unit(1, "cm") )
dev.off()

[1] 108

Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”
Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”
Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”


png 
  2

- sick (Bcl6tg/+, CD70-/-;Bcl6tg/+)

In [138]:
n_sick_single_cells<- rownames(B_cell@meta.data  %>% filter(age == "sick", genotype == "Bcl6tg/+"))
n_sick_double_cells<- rownames(B_cell@meta.data  %>% filter(age == "sick", genotype == "CD70-/-;Bcl6tg/+"))

B_cell_mtx<- GetMatrixFromSeurat2(B_cell, B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_sick_single<- GetMatrixFromSeurat2(B_cell[,n_sick_single_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_mtx_sick_double<- GetMatrixFromSeurat2(B_cell[,n_sick_double_cells], B_cell_genes, 
                                          group = "new_annotation")

B_cell_combined<- cbind(B_cell_mtx_sick_single$exp_mat,
                        B_cell_mtx_sick_double$exp_mat)

B_cell_combined_scaled<- t(apply(B_cell_combined, 1, scale))
B_cell_combined_scaled<- B_cell_combined_scaled  %>% replace(is.na(.),0)

ncol(B_cell_combined_scaled)
B_cell_mtx_sick_single_scaled<-  B_cell_combined_scaled[, 1:27]
B_cell_mtx_sick_double_scaled<-  B_cell_combined_scaled[, 28:54]

colnames(B_cell_mtx_sick_single_scaled)<- colnames(B_cell_mtx$exp_mat)
colnames(B_cell_mtx_sick_double_scaled)<- colnames(B_cell_mtx$exp_mat)

pdf("./results/figures/5_B_cell_dotplot_final_annotation_sick_by_genotype.pdf", width = 15, height = 15)

hp1<- MakeClusterDotPlot(exp_mat = B_cell_mtx_sick_single_scaled, 
               percent_mat = B_cell_mtx_sick_single$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "sick Bcl6tg/+",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha, 
               show_row_dend =FALSE,
               show_column_dend = FALSE,
               legend_dot_size = unit(2, "mm"),
               column_names_side = "top",  cluster_columns = FALSE)

hp2<- MakeClusterDotPlot(exp_mat = B_cell_mtx_sick_double_scaled, 
               percent_mat = B_cell_mtx_sick_double$percent_mat, 
               col_fun = col_fun, cluster_id = cluster_id,
               column_title = "sick CD70-/-;Bcl6tg/+",
               column_title_side = "bottom",         
               row_km = 6,
               row_names_font_size = 8, 
               column_ha = column_ha, row_order = row_order(hp1) %>% unlist(use.names = FALSE),
               legend_dot_size = unit(2, "mm"),
               column_names_side = "top",  cluster_columns = FALSE)

draw(hp1 + hp2 , annotation_legend_list = lgd, ht_gap = unit(1, "cm") )
dev.off()

[1] 54

Warning message:
“The heatmap has not been initialized. You might have different results
if you repeatedly execute this function, e.g. when row_km/column_km was
set. It is more suggested to do as `ht = draw(ht); row_order(ht)`.”


png 
  2